# Purpose

Arrange a script to calculate a Fourier modulus

## install

In [1]:
%pip install -q git+https://github.com/Surpris/BraggPy.git

Note: you may need to restart the kernel to use updated packages.


## setup modules

In [1]:
import h5py
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import os
import braggpy
import time

%matplotlib inline

In [2]:
def load_input_parameter(fpath: str) -> dict:
    with open(fpath, 'r') as ff:
        return json.loads(ff.read())

In [3]:
def generate_lattice_coordinates(parameters: dict) -> np.ndarray:
    prev_number_of_atoms: int = 0
    coors_origin: np.ndarray = None
    coors_inside: np.ndarray = None
    target = parameters.get('target')
    for ind_max in range(10, 50):
        coors_origin = braggpy.make_lattice_points(
            target['unit_cell_length']['value'],
            lattice_type=target['crystal_structure']['value'],
            ind_min=-ind_max, ind_max=ind_max
        )
        coors_inside = braggpy.is_inside(
            coors_origin,
            target['crystal_characteristic_length']['value'],
            target['crystal_shape']['value']
        )
        if prev_number_of_atoms == coors_inside.shape[0]:
            break
        prev_number_of_atoms = coors_inside.shape[0]
    return coors_inside

In [4]:
def generate_momentum(parameters: dict) -> dict:
    reciprocal_lattice = parameters['momentum_space']
    return braggpy.generate_momentum(
        hv,
        reciprocal_lattice['momentum_max']['value'],
        reciprocal_lattice['momentum_step']['value'],
    )

In [5]:
def calculate_euler_angle(parameters: dict, momentum_coors: dict) -> np.ndarray:
    target = parameters['target']
    euler = np.rad2deg(braggpy.calc_euler_hkl(
        target['unit_cell_length']['value'],
        momentum_coors['k0'],
        *target['miller_index']['value']
    ))
    return euler

In [6]:
def add_params(group, key, values):
    if values.get('value'):
        _ = group.create_dataset(key, data=values['value'])
        return group
    group_ = group.create_group(f'{key}')
    for key_, values_ in values.items():
        group_ = add_params(group_, key_, values_)
    return group


def save_results(fpath, parameters, euler, momentum_coors, fourier_modulus, coors):
    with h5py.File(dstpath, 'w') as tree:
        # add input parameters
        tree = add_params(tree, 'input_parameters', parameters)

        # add output
        group = tree.create_group('outputs')
        sub_group = group.create_group('target')
        _ = sub_group.create_dataset('euler_angle', data=euler)

        sub_group = group.create_group('momentum_space')
        _ = sub_group.create_dataset('momentum_x', data=momentum_coors['qxx'])
        _ = sub_group.create_dataset('momentum_y', data=momentum_coors['qyy'])
        _ = sub_group.create_dataset('momentum_z', data=momentum_coors['qzz'])

        sub_group = group.create_group('incident_xray_beam')
        _ = sub_group.create_dataset('wavenumber', data=momentum_coors['k0'])

        sub_group = group.create_group('main_outputs')
        _ = sub_group.create_dataset('fourier_modulus', data=fourier_modulus)
        _ = sub_group.create_dataset('coordinates_of_atoms', data=coors)

## directory setting

In [10]:
outputdir = '../output_data/01_simulation'

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

## calculation with `param.json`

In [14]:
%%time

inputpath = '../input_data/param.json'
fname = 'fourier_modulus_R{0:.2f}A.h5'

props = np.arange(1.0, 0.0, -0.1)

# load parameters
parameters: dict = load_input_parameter(inputpath)
R0: float = parameters['target']['crystal_characteristic_length']['value'] * 1.0

hv: float = None
if parameters['incident_xray_beam'].get('photon_energy'):
    hv = parameters['incident_xray_beam'].get('photon_energy')['value']
else:
    hv = 12.3849 / parameters['incident_xray_beam'].get('wavelength')['value']

# generate momentum coordinates
momentum_coors = generate_momentum(parameters)

# main loop
st = time.time()
for p in props:
    # update characteristic length
    parameters['target']['crystal_characteristic_length']['value'] = R = R0 * p
    dstpath = os.path.join(outputdir, fname.format(R))

    # generate lattice coors
    coors_inside = generate_lattice_coordinates(parameters)

    # Euler rotation
    euler = calculate_euler_angle(parameters, momentum_coors)
    coors_euler = braggpy.euler_rotate(
        coors_inside, euler, 1
    )
    print(dstpath, R, coors_euler.shape)
    # continue

    # calculate modulus
    F = braggpy.calc_modulus(
        coors_euler,
        momentum_coors['qxx'], momentum_coors['qyy'], momentum_coors['qzz']
    )

    # save modulus
    save_results(dstpath, parameters, euler, momentum_coors, F, coors_euler)
    
    print(f"elapsed time: {time.time() - st:.2f} sec.")

../output_data/01_simulation/fourier_modulus_R52.00A.h5 52.0 (10149, 3)
elapsed time: 97.45 sec.
../output_data/01_simulation/fourier_modulus_R46.80A.h5 46.800000000000004 (7419, 3)
elapsed time: 168.61 sec.
../output_data/01_simulation/fourier_modulus_R41.60A.h5 41.6 (5281, 3)
elapsed time: 219.13 sec.
../output_data/01_simulation/fourier_modulus_R36.40A.h5 36.400000000000006 (3511, 3)
elapsed time: 252.39 sec.
../output_data/01_simulation/fourier_modulus_R31.20A.h5 31.200000000000003 (2171, 3)
elapsed time: 273.03 sec.
../output_data/01_simulation/fourier_modulus_R26.00A.h5 26.000000000000007 (1253, 3)
elapsed time: 284.67 sec.
../output_data/01_simulation/fourier_modulus_R20.80A.h5 20.800000000000008 (675, 3)
elapsed time: 291.05 sec.
../output_data/01_simulation/fourier_modulus_R15.60A.h5 15.600000000000009 (249, 3)
elapsed time: 293.41 sec.
../output_data/01_simulation/fourier_modulus_R10.40A.h5 10.40000000000001 (79, 3)
elapsed time: 294.16 sec.
../output_data/01_simulation/fouri

## calculation with `param_2.json`

In [15]:
%%time

inputpath = '../input_data/param_2.json'
fname = 'fourier_modulus_R{0:.2f}A.h5'

props = np.arange(1.0, 0.4, -0.1)

# load parameters
parameters: dict = load_input_parameter(inputpath)
R0: float = parameters['target']['crystal_characteristic_length']['value'] * 1.0

hv: float = None
if parameters['incident_xray_beam'].get('photon_energy'):
    hv = parameters['incident_xray_beam'].get('photon_energy')['value']
else:
    hv = 12.3849 / parameters['incident_xray_beam'].get('wavelength')['value']

# generate momentum coordinates
momentum_coors = generate_momentum(parameters)

# main loop
st = time.time()
for p in props:
    # update characteristic length
    parameters['target']['crystal_characteristic_length']['value'] = R = R0 * p
    dstpath = os.path.join(outputdir, fname.format(R))

    # generate lattice coors
    coors_inside = generate_lattice_coordinates(parameters)

    # Euler rotation
    euler = calculate_euler_angle(parameters, momentum_coors)
    coors_euler = braggpy.euler_rotate(
        coors_inside, euler, 1
    )
    print(dstpath, R, coors_euler.shape)
    # continue

    # calculate modulus
    F = braggpy.calc_modulus(
        coors_euler,
        momentum_coors['qxx'], momentum_coors['qyy'], momentum_coors['qzz']
    )

    # save modulus
    save_results(dstpath, parameters, euler, momentum_coors, F, coors_euler)
    
    print(f"elapsed time: {time.time() - st:.2f} sec.")

../output_data/01_simulation/fourier_modulus_R112.00A.h5 112.0 (102417, 3)
elapsed time: 1001.21 sec.
../output_data/01_simulation/fourier_modulus_R100.80A.h5 100.8 (74605, 3)
elapsed time: 1743.60 sec.
../output_data/01_simulation/fourier_modulus_R89.60A.h5 89.60000000000001 (52661, 3)
elapsed time: 2260.68 sec.
../output_data/01_simulation/fourier_modulus_R78.40A.h5 78.4 (35241, 3)
elapsed time: 2603.43 sec.
../output_data/01_simulation/fourier_modulus_R67.20A.h5 67.20000000000002 (21913, 3)
elapsed time: 2815.56 sec.
../output_data/01_simulation/fourier_modulus_R56.00A.h5 56.000000000000014 (12695, 3)
elapsed time: 2938.35 sec.
CPU times: user 49min 3s, sys: 38.2 s, total: 49min 41s
Wall time: 48min 58s


In [16]:
for fname_ in os.listdir(outputdir):
    print(os.path.join(outputdir, fname_))

../output_data/01_simulation/fourier_modulus_R41.60A.h5
../output_data/01_simulation/fourier_modulus_R89.60A.h5
../output_data/01_simulation/fourier_modulus_R78.40A.h5
../output_data/01_simulation/fourier_modulus_R10.40A.h5
../output_data/01_simulation/fourier_modulus_R31.20A.h5
../output_data/01_simulation/fourier_modulus_R56.00A.h5
../output_data/01_simulation/fourier_modulus_R5.20A.h5
../output_data/01_simulation/fourier_modulus_R46.80A.h5
../output_data/01_simulation/fourier_modulus_R52.00A.h5
../output_data/01_simulation/fourier_modulus_R67.20A.h5
../output_data/01_simulation/fourier_modulus_R112.00A.h5
../output_data/01_simulation/fourier_modulus_R26.00A.h5
../output_data/01_simulation/fourier_modulus_R36.40A.h5
../output_data/01_simulation/fourier_modulus_R20.80A.h5
../output_data/01_simulation/fourier_modulus_R15.60A.h5
../output_data/01_simulation/fourier_modulus_R100.80A.h5
